# **Loading data**

In [2]:
import pandas as pd
import numpy as np
import cv2
import skimage.color
from PIL import Image

def read_data(file_path):
    images = []
    labels = []
    
    path = '../input/mars-surface-and-curiosity-image-set-nasa/Mars Surface and Curiosity Image/'
    f1 = open(file_path,'r')
    for line in f1:
        filename, label = line.split(' ')
        #print(label,filename)
        #filename = filename[11:]
        path1 = path+filename
        #print(path1)
        img = cv2.imread(path1)
        #img = rgb2gray(imread(path1))
        #img = Image.open(path1).convert('LA')
        #img.save()
        img = cv2.resize(img, (32,32))
        images.append(img)
        labels.append(int(label))

    images = np.array(images)
    labels = np.array(labels)
    return images,labels,images,labels

x_train, y_train,images, labels = read_data('train_file.txt')
x_test, y_test, images, labels= read_data('test_file.txt')

#print(y_train[5])

# Label distribution

In [4]:
import matplotlib.pyplot as plt
plt.hist(labels,edgecolor = 'black',bins = 24)
plt.title("mars surface")
plt.xlabel("labels")
plt.ylabel("frequency")
plt.show()

# Finding best 'n' for KNN

In [5]:
neigh = np.arange(3,7)
train_accuracy = np.empty(len(neigh))
test_accuracy = np.empty(len(neigh))

for i,j in enumerate(neigh):
    knn = KNeighborsClassifier(n_neighbors = j)
    knn.fit(x_train,y_train)
    train_accuracy[i] = knn.score(x_train,y_train)
    test_accuracy[i] = knn.score(x_test,y_test)
    
plt.title('knn accuracy')
plt.plot(neigh, test_accuracy, label = 'Testing Accuracy')
plt.plot(neigh, train_accuracy, label = 'Training Accuracy')
plt.legend()
plt.xlabel('Number of Neighbors')
plt.ylabel('Accuracy')
plt.show()

# Decision Tree

In [20]:
from sklearn.tree import DecisionTreeClassifier
import seaborn as sns

clf = DecisionTreeClassifier(max_depth = 21)
clf.fit(x_train,y_train)
predict_dt = clf.predict(x_test)
cm = confusion_matrix(y_test,predict_dt)
print(classification_report(y_test,predict_dt))
print(clf.score(x_test,y_test))

plt.rcParams['figure.figsize'] = (10,10)
sns.heatmap(cm, annot=True)
plt.title("Confusion Matrix", fontsize = 15)
plt.show()


In [8]:
!pip install pydotplus

In [9]:
from six import StringIO
from IPython.display import Image
from sklearn.tree import export_graphviz
import pydotplus
import graphviz

features = []
dot_data = StringIO()
export_graphviz(clf, out_file=dot_data,  
                filled=True, rounded=True,
                special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_png('mars.png')
graph.set_size('"100,5!"')
Image(graph.create_png())

# SVM

In [1]:
from sklearn import svm

clf = svm.SVC(decision_function_shape='ovo')
clf.fit(x_train, y_train)
predict_svm = clf.predict(x_test)
cm = confusion_matrix(y_test,predict_svm)
print(classification_report(y_test,predict_svm))
print(clf.score(x_test,y_test))

plt.rcParams['figure.figsize'] = (10,10)
sns.heatmap(cm, annot=True)
plt.title("Confusion Matrix", fontsize = 15)
plt.show()

z = lambda x,y: (-clf.intercept_[0]-clf.coef_[0][0]*x -clf.coef_[0][1]*y) / clf.coef_[0][2]
tmp = np.linspace(-5,5,30)
x,y = np.meshgrid(tmp,tmp)
fig = plt.figure()
ax  = fig.add_subplot(111, projection='3d')
ax.plot3D(x_train[y_train==0,0], x_train[y_train==0,1], x_train[y_train==0,2],'ob')
ax.plot3D(x_train[y_train==1,0], x_train[y_train==1,1], x_train[y_train==1,2],'sr')
ax.plot_surface(x, y, z(x,y))
ax.view_init(30, 60)
plt.show()

NameError: name 'x_train' is not defined

# Random Forest

In [24]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=50,random_state = 33)
rf.fit(x_train, y_train)
predict_rf = rf.predict(x_test)
print(rf.score(x_test,y_test))

